# Report generation

In [2]:
from datetime import datetime

import math
import pickle
from os import listdir
from os.path import isfile, join
import os
import csv

from flask import Flask, render_template, request
import tensorflow as tf
from tensorflow import keras
import copy
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from numpy import concatenate
import warnings
from tqdm import tqdm

In [3]:
def get_station_names():
    mypath = '../datasets/bss/dublin/ml_models/'
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

    station_ids = [x.split('.')[0].split('_')[1] for x in files]

    output = {}

    for sid in station_ids:
        csv_file = csv.reader(open('../datasets/bss/dublin/original/dublin.csv', "r"), delimiter=",")
        for row in csv_file:
            if sid == row[0]:
                output[sid] = row[1]

    return output


def simple_predict(station_id, int_time, int_date, int_day):
    destination_directory = '../datasets/bss/dublin/simple_ml_models/'
    scaler_destination_directory = copy.deepcopy(destination_directory) + 'scalers/'

    model = tf.keras.models.load_model(destination_directory + 'station_' + str(station_id) + '.h5')

    file = open(scaler_destination_directory + 'station_' + str(station_id) + '.pkl', "rb")
    scaler = pickle.load(file)
    file.close()

    params = np.array([0, int_time, int_date, int_day])
    params = params.reshape(1, -1)
    params = scaler.transform(params)
    params = np.array([params])
    params = params.tolist()
    params[0][0].pop(0)
    params = np.array(params)

    answer = model.predict(params)
    full_row = concatenate((answer, params[0]), axis=1)
    inv_row = scaler.inverse_transform(full_row)
    
    return inv_row[0][0]

In [4]:
def generate_report(date=datetime(year=2019, month=8, day=1)):
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    destination_directory = '../outputs/date_reports/'
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)
    
    
    with open(destination_directory + date.strftime("%Y-%m-%d") +  '.csv', 'w', newline='') as csvfile:
        times = ["station"]
        for x in range(24):
            times.append(str(x) + ":00")
        
        writer = csv.DictWriter(csvfile, fieldnames=times)

        writer.writeheader()
        
        stations = get_station_names()
        
        for sid, name in tqdm(stations.items()):
            answers = {"station":sid}
            for x in range(24):
                answers[str(x) + ":00"] = (
                    round(
                        simple_predict(
                            sid,
                            (x * 12),
                            date.strftime('%j'),
                            date.strftime('%w')
                        )
                    )
                )
            writer.writerow(answers)
            

generate_report(datetime(year=2019, month=12, day=2))

 57%|██████████████████████████████████████████████▏                                  | 61/107 [55:52<42:08, 54.97s/it]


KeyboardInterrupt: 